In [1]:
!gdown 1-M68LD0_1mREYHzhVaxpy8n0soXaUK0n

Downloading...
From: https://drive.google.com/uc?id=1-M68LD0_1mREYHzhVaxpy8n0soXaUK0n
To: /content/Приказ Минтранса России от 23 06 2022 N 250.docx
100% 5.30M/5.30M [00:00<00:00, 92.3MB/s]


In [ ]:
!pip install transformers sentence-transformers openpyxl\
accelerate langchain docx2txt chromadb bitsandbytes peft

In [1]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

import transformers
import peft
import torch

from tqdm.autonotebook import tqdm as notebook_tqdm

## Embedding the dataset

In [2]:
embed_model_id = 'intfloat/multilingual-e5-base'

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [3]:
raw_documents = Docx2txtLoader('Приказ Минтранса России от 23 06 2022 N 250.docx').load()
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1000, chunk_overlap=0)

documents = text_splitter.split_documents(raw_documents)

In [4]:
db = Chroma.from_documents(documents, embed_model)

In [5]:
retriever = db.as_retriever(search_type="mmr")

## LLM

In [6]:
model_id = "IlyaGusev/saiga2_13b_lora"

hf_auth = 'TOKEN'

model_config = peft.PeftConfig.from_pretrained(model_id)

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_config.base_model_name_or_path,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

model = peft.PeftModel.from_pretrained(
    model,
    model_id,
    torch_dtype=torch.float16
).eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [10]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.1,
    do_sample=True,
    max_new_tokens=512,
    repetition_penalty=1.1,
    num_beams=1,
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

## RetrievalQA

In [11]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [12]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
        llm=llm, chain_type='stuff',
        retriever=retriever
    )

In [14]:
rag_pipeline

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ab410685d80>)), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7ab463d326b0>, search_type='mmr'))

In [15]:
question_template = "system:\n"\
                    "Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце. "\
                    "Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ."\
                    "\n\n{context}\n\nВопрос: {question}\nОтвет: "

rag_pipeline.combine_documents_chain.llm_chain.prompt.template = question_template

In [16]:
rag_pipeline

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='system:\nИспользуй следующие фрагменты контекста, чтобы ответить на вопрос в конце. Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ.\n\n{context}\n\nВопрос: {question}\nОтвет: '), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ab410685d80>)), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7ab463d326b0>, search_type='mmr'))

---

## Check

In [ ]:
rag_pipeline('Кто должен выполнять правила технической эксплуатации?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'query': 'Кто должен выполнять правила технической эксплуатации?',
 'result': ' Владельцы инфраструктуры (владельцы железнодорожных путей необщего пользования) и дежурные по железнодорожным станциям.'}

In [ ]:
rag_pipeline('На ком ответственность за содержание путей?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'query': 'На ком ответственность за содержание путей?',
 'result': ' Ответственность за содержание путей лежит на владельце железнодорожных путей необщего пользования.'}

In [ ]:
rag_pipeline('Кому разрешен доступ на локомотивы?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'query': 'Кому разрешен доступ на локомотивы?',
 'result': ' Разрешение на доступ на локомотивы может быть предоставлено только владельцем инфраструктуры (владельцем железнодорожных путей необщего пользования).'}

---

In [17]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [22]:
!gdown 1nJXpYdurC1q8jvGwoDggWPrI4aWb6Roi

Downloading...
From: https://drive.google.com/uc?id=1nJXpYdurC1q8jvGwoDggWPrI4aWb6Roi
To: /content/output.xlsx
100% 352k/352k [00:00<00:00, 171MB/s]


In [18]:
import pandas as pd

data = pd.read_excel('output.xlsx')
data = data[['question', 'answer_summary', "answers_merged"]]

In [19]:
random_row = data.sample()
print(random_row)
index = random_row.index[0]
name_value = random_row['question'].iloc[0]
print(index, name_value)

                                               question  \
1022  Как выглядит подталкивающий локомотив или спец...   

                                         answer_summary  \
1022  Подталкивающий локомотив и специальный самоход...   

                                         answers_merged  
1022  ['Подталкивающий локомотив и самоходный специа...  
1022 Как выглядит подталкивающий локомотив или специальный самоходный подвижной состав?


In [20]:
import random

from sentence_transformers import SentenceTransformer, util


def get_question(data):
    shape = data.shape[0]
    index = random.randint(0, shape)
    question = data.iloc[index]['question']
    return index, question


def init_model(model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
    model = SentenceTransformer(model_name)
    return model


def get_score_from_answers(model, question_id, user_answer):
    expected_answer_summary = data.iloc[question_id]['answer_summary']
    expected_answers = eval(data.iloc[question_id]['answers_merged'])
    embeddings1 = model.encode([user_answer], convert_to_tensor=True)
    embeddings2 = model.encode(expected_answers, convert_to_tensor=True)
    cosine_score = float(util.cos_sim(embeddings1, embeddings2).max().item())
    print(user_answer, *expected_answers, cosine_score, sep='\n')
    return cosine_score, expected_answer_summary


message_template = "{role}\n{content}\n"

def answerUsingKnowledgeBase(question):
    answer = rag_pipeline(question)
    return answer['result'].strip()


def answerUsingHistory(history_json : list[dict]):
    history = ""

    for message in history_json:
        message_text = message_template.format(**message)
        history += message_text

    answer = llm(history)

    return answer.strip()


def generateAnswer(history_json : list[dict]):
    assert len(history_json) >= 2

    if len(history_json) == 2:
        question = history_json[-1]["content"]
        return answerUsingKnowledgeBase(question)

    return answerUsingHistory(history_json)

In [ ]:
!pip install uvicorn nest_asyncio fastapi pyngrok kaleido python-multipart pydantic

In [26]:
!ngrok config add-authtoken TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import os
import sys
import json
import uvicorn
import nest_asyncio
import time
import re

from fastapi import FastAPI, File, UploadFile, Request, Response, HTTPException, Form
from fastapi.templating import Jinja2Templates
from fastapi.responses import FileResponse

from pydantic import BaseModel
from pyngrok import ngrok
from PIL import Image

from typing import List, Dict

testing_model = init_model()

class Question(BaseModel):
    context: List[Dict[str, str]]


class AnswerFromUser(BaseModel):
    question_id: int
    answer: str


app = FastAPI()


@app.get('/')
async def main(request: Request):
    return {"message": "Hello World"}


@app.get('/get_rand_question')
async def get_rand_question(request: Request):
    question_id, question = get_question(data)
    responce = {
        "question_id": question_id,
        "question": question
    }
    return responce


@app.post("/get_answer")
async def get_answer(question: Question):
    answer = generateAnswer(question.context)
    print(question.context)
    return answer


@app.post("/check_answer")
async def check_answer(answer_from_user: AnswerFromUser):
    question_id = answer_from_user.question_id
    user_answer = answer_from_user.answer
    answer_coef, expected_answer = get_score_from_answers(testing_model, question_id, user_answer)
    print(answer_coef, user_answer, )
    response = {
        "answer_coef": answer_coef,
        "expected_answer": expected_answer
    }
    return response


ngrok_tunnel = ngrok.connect(8000,  domain="patient-buck-weekly.ngrok-free.app")
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [37365]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://patient-buck-weekly.ngrok-free.app
[{'role': 'system', 'content': 'Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце. Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ'}, {'role': 'user', 'content': 'Какие требования предъявляются к размещению сооружений и устройств вагонного хозяйства?'}]
INFO:     188.64.15.38:0 - "POST /get_answer HTTP/1.1" 200 OK
[{'role': 'system', 'content': 'Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце. Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ'}, {'role': 'user', 'content': 'Какие требования предъявляются к размещению сооружений и устройств вагонного хозяйства?'}, {'role': 'bot', 'content': 'Требования к размещению сооружений и устройств вагонного хозяйства определяются правилами технического обслуживания железнодорожного транспорта и другими нормативными документами. Они включают в себя требо